<a href="https://colab.research.google.com/github/shiehn/rune_notebooks/blob/main/elixir_demucs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Check if runes-client is installed, if not then install it
try:
    import runes_client
except ImportError:
    !pip install runes-client
    import runes_client

# Check if demucs is installed, if not then install it
try:
    import demucs
except ImportError:
    !pip install demucs
    import demucs

# Check if ffmpeg is installed, if not then install it
ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg


import runes_client.core as runes
from runes_client import RunesFilePath

import os
import subprocess
import stat
import time

placeholder_txt = "Enter the token generated by the Crucible plugin"
RUNES_TOKEN = "34c7434b-4620-4972-af46-77e4972d4681" #@param {type:"string"}
runes_token = RUNES_TOKEN

if runes_token is None or runes_token == "" or runes_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

async def runes_func(input_file: RunesFilePath):
    try:
        command = ['python3', '-m', 'demucs']

        print(f"input_file: {input_file}")

        # Add the audio file path
        command.append(input_file)

        # Execute the command
        subprocess.run(command)

        model = 'htdemucs'
        input_file_without_extension = os.path.splitext(os.path.basename(input_file))[0]
        path = os.path.join('separated', model, input_file_without_extension)

        files = os.listdir(path)
        for file in files:
            full_path = os.path.join(path, file)

            await runes.output().add_file(full_path)

        await runes.output().add_message("Stem Seperation Complete")

        return True
    except Exception as e:

        await runes.output().add_error(f"Method encountered an error: {e}")
        return False


runes.set_token(token=runes_token)
runes.set_name("Stem Separation - DEMUCS")
runes.set_description("Send an audio file & have the Rune split stems and return them")
runes.register_method(runes_func)

runes.set_output_target_format('wav')
runes.set_output_target_channels(2)
runes.set_output_target_sample_rate(44100)
runes.set_output_target_bit_depth(16)

runes.connect_to_server()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 M

CREATING CONTRACT {'method_name': 'runes_func', 'params': [{'name': 'input_file', 'type': 'RunesFilePath', 'default_value': None, 'ui_component': None}], 'author': 'Default Author', 'name': 'Stem Separation - DEMUCS', 'description': 'Send an audio file & have the Rune split stems and return them', 'version': '0.0.0'}
PENDING REQUESTS: []
Heartbeat successful.
Successfully created compute contract. Response: {"id":"3746329a-b96c-53ea-a569-245dc93af304","data":{"method_name":"runes_func","params":[{"name":"input_file","type":"RunesFilePath","default_value":null,"ui_component":null}],"author":"Default Author","name":"Stem Separation - DEMUCS","description":"Send an audio file & have the Rune split stems and return them","version":"0.0.0"},"created_at":"2024-04-09T06:11:56","updated_at":"2024-04-09T06:11:56"}
ADD_MAPPING_RES: {"id":"c6d85dfa-fbfe-48bc-851b-837ced9a4b90","master_token":"34c7434b-4620-4972-af46-77e4972d4681","connection_token":"3746329a-b96c-53ea-a569-245dc93af304","connecti

KeyboardInterrupt: 